In [1]:
import sys
sys.path.append('C:/Users/juans/OneDrive/Documentos/iaProyect/Cognitive')
print(sys.path)  # Esto mostrará todas las rutas en sys.path

from model.preprocessing.data_cleaning import DataCleaner
from model.utils.data_analysis import DataAnalyzer
#from model.preprocessing.data_extraction import DataExtractor
from model.preprocessing.feature_extraction import FeatureSelector,FeatureVisualizer



['C:\\Users\\juans\\AppData\\Local\\Programs\\Python\\Python312\\python312.zip', 'C:\\Users\\juans\\AppData\\Local\\Programs\\Python\\Python312\\DLLs', 'C:\\Users\\juans\\AppData\\Local\\Programs\\Python\\Python312\\Lib', 'C:\\Users\\juans\\AppData\\Local\\Programs\\Python\\Python312', 'c:\\Users\\juans\\OneDrive\\Documentos\\iaProyect\\Cognitive\\.venv', '', 'c:\\Users\\juans\\OneDrive\\Documentos\\iaProyect\\Cognitive\\.venv\\Lib\\site-packages', 'c:\\Users\\juans\\OneDrive\\Documentos\\iaProyect\\Cognitive\\.venv\\Lib\\site-packages\\win32', 'c:\\Users\\juans\\OneDrive\\Documentos\\iaProyect\\Cognitive\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\juans\\OneDrive\\Documentos\\iaProyect\\Cognitive\\.venv\\Lib\\site-packages\\Pythonwin', 'C:/Users/juans/OneDrive/Documentos/iaProyect/Cognitive']


In [1]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
import unidecode
import csv

class DataExtractor:
    def __init__(self, base_path="../../data/", columns_to_drop=None):
        self.base_path = base_path
        self.columns_to_drop = columns_to_drop if columns_to_drop is not None else []

    def preprocess_csv(self, input_path, output_path, encoding='latin1'):
        with open(input_path, 'r', encoding=encoding) as infile, \
            open(output_path, 'w', encoding=encoding) as outfile:
            first_line = True
            buffer = ''
            for line in infile:
                if first_line:
                    # Escribe la primera línea directamente para evitar manipular los nombres de las columnas
                    outfile.write(line.replace('//', ''))
                    first_line = False
                elif '//' in line:
                    # Procesa la línea solo si se encuentra el delimitador
                    parts = line.split('//')
                    # Completa el buffer actual antes de empezar uno nuevo
                    buffer += ' ' + parts[0].strip()
                    outfile.write(buffer + '\n')  # Asegúrate de no cambiar los delimitadores internos si no es necesario
                    buffer = parts[1].strip() if len(parts) > 1 else ''
                else:
                    buffer += ' ' + line.strip()
            if buffer:
                # Asegúrate de escribir cualquier dato restante en el buffer
                outfile.write(buffer + '\n')


    def load_data(self, nameFolderRaw, file_name, processed_file_name='processed.csv', encoding='latin1', delimiter=';'):
        original_path = self.base_path + nameFolderRaw + file_name
        processed_path = self.base_path + nameFolderRaw + processed_file_name
        self.preprocess_csv(original_path, processed_path, encoding=encoding)
        
        try:
            data = pd.read_csv(processed_path, 
                               encoding=encoding, 
                               delimiter=delimiter, 
                               header=0,
                               quoting=csv.QUOTE_NONE)
        except pd.errors.ParserError as e:
            print(f"Error al leer el archivo: {e}")
            return None

        if self.columns_to_drop:
            data.drop(columns=self.columns_to_drop, inplace=True, errors='ignore')

        return data

# Ejemplo de uso
datahandler = DataExtractor()
data = datahandler.load_data("raw/", "DB HALF.csv")
if data is not None:
    print(data.head())
else:
    print("No se pudo cargar los datos.")





   Identificaciï¿½n  Lugar Nacimiento  Municipio Atenciï¿½n  Edad  Gï¿½nero  \
0          10035213                 0                     1    63         0   
1          10055205                 2                     3    79         0   
2          10056901                 3                     3    73         0   
3          10061985                 6                     3    71         0   
4          10065630                 6                     3    69         0   

   Grupo Sanguï¿½neo  Lateralidad  Autopercepciï¿½n Salud  Estado Civil  \
0                  7            1                       1             1   
1                  6            1                       1             0   
2                  7            1                       1             1   
3                  0            1                       1             2   
4                  0            1                       0             1   

   Vive con pareja  ...  Cdr Juicio  Cdr Vida Social  Cdr Hogar Y Aficione

C:\Users\juans\AppData\Local\Temp\ipykernel_29764\3458530491.py:42: DtypeWarning: Columns (44,67) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(processed_path,


In [3]:

import numpy as np
print(np.shape(data))
print(data.info())


(2986, 269)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2986 entries, 0 to 2985
Columns: 269 entries, Identificaciï¿½n to limiter
dtypes: float64(5), int64(216), object(48)
memory usage: 6.1+ MB
None


In [ ]:


import pandas as pd
import numpy as np



# Instanciar y usar el preprocesador
preprocessor = DataCleaner(X, y)
data_analyzer = DataAnalyzer()


preprocessor.encode_features(target_column='income', method='onehot')

# Limpieza de duplicados y datos nulos
preprocessor.delete_null_data()
#preprocessor.delete_duplicate_rows()
#preprocessor.delete_Object_columns()

# Imputación de valores faltantes
#X_imputed = preprocessor.impute_missing_values()

# Estandarización
preprocessor.standardize()

# normalización
preprocessor.normalize_features()

#detectar y eliminar outliers
preprocessor.delete_outliers().value_counts()

#balancear datos
preprocessor.balance_data()

#generar columnas polinomiales
#xfold = preprocessor.generate_polynomial_columns()

#preprocessor.reduce_dimensionality(n_components=2,replace=True)

data = pd.concat([preprocessor.X, preprocessor.y], axis=1)
print(data_analyzer.describe_data(data=data))
data_analyzer.visualize_correlation(data=data)
data_analyzer.visualize_data(data, kind='boxplot')
data_analyzer.visualize_data(data, kind='histogram')




In [ ]:
X = preprocessor.X
y = preprocessor.y


# Inicializar el selector de características
selector = FeatureSelector(X, y)

# 1. Selección de características con Chi-squared
chi2_result = selector.select_features_chi2(k=14)
FeatureVisualizer.plot_scores(chi2_result['features'], chi2_result['scores'], title='Chi-squared Scores')

# 2. Selección de características con Spearman
spearman_result = selector.select_features_spearman(threshold=0.01)
FeatureVisualizer.plot_correlations(spearman_result['selected_features'], spearman_result['correlations'], title='Spearman Correlation Coefficients')

# 3. Selección de características con Lasso
lasso_result = selector.lasso_feature_selection(alpha=0.01)
FeatureVisualizer.plot_coefficients(lasso_result['selected_features'], lasso_result['coefficients'], title='Lasso Coefficients')

# 4. Selección de características con Sequential Feature Selector
sfs_result = selector.sequential_feature_selection(n_features_to_select=13, direction='forward')
print("Sequential Feature Selection Selected Features:", sfs_result['selected_features'])

# 5. Importancia de características con Random Forest
rf_result = selector.random_forest_feature_importance(n_features=14)
FeatureVisualizer.plot_importances(rf_result['selected_features'], rf_result['importances'], title='Random Forest Feature Importances')